In [42]:
"""
载入需要用到的包和数据
"""
%config InlineBackend.figure_format = 'retina'
import idx2numpy
import matplotlib.pyplot as plt
import numpy as np

In [43]:
x_train = idx2numpy.convert_from_file('./DataSets/mnist/train-images.idx3-ubyte')
y_train = idx2numpy.convert_from_file('./DataSets/mnist/train-labels.idx1-ubyte')
x_test = idx2numpy.convert_from_file('./DataSets/mnist/t10k-images.idx3-ubyte')
y_test = idx2numpy.convert_from_file('./DataSets/mnist/t10k-labels.idx1-ubyte')

In [44]:
"""
获得数据，并对因变量进行独热编码
"""

'\n获得数据，并对因变量进行独热编码\n'

In [45]:
np.random.seed(1)

In [46]:
train_images,train_labels = (x_train/255,y_train)

In [47]:
# 训练数据的独热码
one_hot_labels = np.zeros((len(train_labels),10))
for i,j in enumerate(train_labels):
    one_hot_labels[i][j] = 1
    # print(f"i:{i}, j:{j}")
    # 这什么意思？
train_labels = one_hot_labels

In [48]:
"""
把训练数据分成训练数据和验证数据
"""

# 验证数据
valid_images, valid_labels = train_images[-10000:], train_labels[-10000:]
# 训练数据
train_images, train_labels = train_images[:1000], train_labels[:1000]

In [49]:
# 测试数据中的自变量矩阵
test_images = x_test/255
# 测试数据的独热码
test_labels = np.zeros((len(y_test),10))
for i,j in enumerate(y_test):
    test_labels[i][j] = 1

In [50]:
def relu(x):
    return (x>0) * x

def relu2derive(x):
    return (x>0)

def softmax(x):
    temp = np.exp(x)
    return temp/np.sum(temp, axis = 1, keepdims = True)

In [51]:
def get_image_section(layer, row_start, row_end, col_start, col_end):
    section = layer[:,row_start:row_end, col_start:col_end]
    return section.reshape(-1, 1, row_end-row_start, col_end-col_start)

def conv_reshape(image, kernel_rows, kernel_cols):
    image = np.pad(image, ((0,0),(1,1),(1,1)),mode='constant')
    image_sections = []
    for row_start in range(image.shape[1] - kernel_rows + 1):
        for col_start in range(image.shape[2] - kernel_cols + 1):
            image_sections.append(get_image_section(image, row_start, row_start+kernel_rows, col_start, col_start+kernel_cols))
            # 为什么这里没有-1
    expanded_input = np.concatenate(image_sections,axis=1)
    es = expanded_input.shape
    layer = expanded_input.reshape(es[0]*es[1],-1)
    return layer

def conv_2d(layer,kernels,bias):
    layer = np.dot(layer,kernels) + bias
    return relu(layer)

def maxpool(layer, pool_rows, pool_cols, batch_size, input_rows, input_cols, num_kernels):
    layer =layer.reshape(batch_size,input_rows, input_cols, num_kernels)
    
    layer_out_shape = (layer.shape[0], int(layer.shape[1]/2), int(layer.shape[2]/2), layer.shape[3])
    layer_out = np.zeros(layer_out_shape)
    argmax_out = []
    # layer_2_drive = np.zeros(layer.shape)
    for k in range(num_kernels):
        layer_sections = []
        for row_start in range(0, layer.shape[1] - pool_rows + 1, 2):
            for col_start in range(0, layer.shape[2] - pool_cols +1, 2):
                layer_sections.append(get_image_section(layer[:,:,:,k],row_start, row_start+pool_rows, col_start, col_start+pool_cols))
        layer_temp = np.concatenate(layer_sections, axis=1)
        layer_temp = layer_temp.reshape(layer_temp.shape[0]*layer_temp.shape[1], -1)
        out = np.max(layer_temp,axis=1)
        argmax_out.append(np.argmax(layer_temp, axis=1))
        layer_out[:,:,:,k]=out.reshape(layer_out_shape[:-1])
    return layer_out, argmax_out

In [52]:
def maxpool_2_derive(delta, pool_argmax, pool_rows, pool_cols, batch_size, input_rows, input_cols, num_kernels):
    
    delta_conv = np.zeros((batch_size,input_rows,input_cols,num_kernels))
    
    after_pool_rows = int(input_rows/pool_rows)
    after_pool_cols = int(input_cols/pool_cols)
    
    delta = delta.reshape(batch_size,after_pool_rows,after_pool_cols,num_kernels)
    
    for k in range(num_kernels):
        delta_k = np.zeros((int(batch_size*after_pool_rows*after_pool_cols), pool_rows*pool_cols))
        
        delta_k[:,pool_argmax[k]] = delta[:,:,:,k].flatten()
        delta_k = delta_k.re(batch_size, after_pool_rows*after_pool_cols, pool_rows*pool_cols)
        delta_k_row = 0
        for row_start in range(0, input_rows - pool_rows + 1, 2):
            for col_start in range(0,input_cols - pool_cols + 1, 2):
                delta_conv[:,row_start:(row_start+pool_rows),col_start:(col_start+pool_cols),k] = delta_k[:,delta_k_row,:].reshape(batch_size,pool_rows,pool_cols)
                delta_k_row += 1
    return delta_conv

In [53]:
lr, epoches = 0.01, 1000
pixels_per_image, num_labels = 784, 10

input_rows, input_cols = 28, 28
# 卷积层核的宽度高度和深度
kernel_rows,kernel_cols, num_kernels = 3,3,32
pool_rows,pool_cols=2,2
FC_size=128

batch_size = 100
num_batch = int(len(train_images)/batch_size)

hidden_maxpool = int((input_rows/2)*(input_cols/2)*num_kernels)


b_kernels = np.zeros((1,num_kernels))  # 截距初始值全部为0
b_2_3 = np.zeros((1,FC_size))          # b_2_3全部初始为0
b_3_4 = np.zeros((1,num_labels))       # b_3_4全部初始为0

# 卷积核的初始值
kernels = 0.2*np.random.random((kernel_rows*kernel_cols, num_kernels)) - 0.1
w_2_3 = 0.2*np.random.random((hidden_maxpool, FC_size)) - 0.1
w_3_4 = 0.2 * np.random.random((FC_size , num_labels)) -0.1

In [ ]:
for epoch in range(epoches):
    correct_cnt, val_correct_cnt = 0,0
    loss, val_loss = 0.0,0.0
    
    for i in range(num_batch):
        #----------------------------------------------#
        # 正向传播算法
        batch_start, batch_end = i*batch_size, (i-1)*batch_size
        batch_image = train_images[batch_start:batch_end]
        batch_label = train_labels[batch_start:batch_end]
        layer_0 = conv_reshape(batch_image, kernel_rows, kernel_cols)
        layer_1 = conv_2d(layer_0, kernels, b_kernels)
        layer_1_shape = layer_1.shape
        
        layer_2, pool_argmax = maxpool(layer_1, pool_rows, pool_cols, batch_size, input_rows, input_cols, num_kernels)
        layer_2 = layer_2.reshape(batch_size, -1)
        
        layer_3 = relu(np.dot(layer_2, w_2_3) + b_2_3)
        
        layer_4 = softmax(np.dot(layer_3, w_3_4) + b_3_4)
        #-------------------------------------------------#
        # 反向传播算法
        for k in range(batch_size):
            loss -= np.log(layer_4[k:k+1, np.argmax(batch_label[k:k+1])])
            
            correct_cnt += int(np.argmax(layer_4[k:k+1]) == np.argmax(batch_label[k:k+1]))
            
        layer_4_delta = (layer_4 - batch_label)/batch_size
        layer_3_delta = np.dot(layer_4_delta, w_3_4.T)*relu2derive(layer_3)
        
        layer_2_delta = np.dot(layer_3_delta, w_2_3.T)
        layer_2_delta = maxpool_2_derive(layer_2_delta, pool_argmax, pool_rows, pool_cols, batch_size, input_rows,input_cols,num_kernels)
        
        layer_1_delta = layer_2_delta.reshape(layer_1_shape)*relu2derive(layer_1)
        
        b_3_4 -=lr * np.sum(layer_4_delta, axis=0, keepdims=True)
        b_2_3 -=lr * np.sum(layer_3_delta, axis=0, keepdims=True)
        b_kernels -= lr * np.sum(layer_1_delta, axis=0, keepdims=True)/(input_rows*input_cols)
        
        w_3_4 -= lr * layer_3.T.dot(layer_4_delta)
        w_2_3 -= lr * layer_2.T.dot(layer_3_delta)
        kernels -= lr * layer_0.T.dot(layer_1_delta)/(input_rows * input_cols)
        
        #---------------------------------------------------------#
        # 每训练100次，计算一次验证误差
        if(epoch % 100 ==0 or epoch==epoches-1):
            layer_0 = conv_reshape(valid_images, kernel_rows, kernel_cols)
            layer_1 = conv_2d(layer_0, kernels, b_kernels)
            layer_1 = layer_1.reshape(len(valid_images), -1)
            
            layer_2, pool_argmax  = maxpool(layer_1,pool_rows, pool_cols, len(valid_images), input_rows, input_cols, num_kernels)
            
            layer_3 = relu(np.dot(layer_2.reshape(len(valid_images),-1),w_2_3)+b_2_3)
            
            layer_4 = softmax(np.dot(layer_3, w_3_4) + b_3_4)
            
            for k in range(len(valid_images)):
                val_loss -= np.log(layer_4[k:k+1,np.argmax(valid_labels[k:k+1])])
                val_correct_cnt += int(np.argmax(layer_4[k:k+1]) == np.argmax(valid_labels[k:k+1]))
                
            print("e:%3d; Tr_Loss:%0.2f; Tr_Acc:%0.2f; Val_Loss: %0.2f; Val_Acc:%0.3f"%(epoch, loss/float(len(train_images)),correct_cnt/float(len(train_images)), val_loss/float(len(train_images)), val_correct_cnt/float(len(train_images))))
        

In [ ]:
#-------------------------------------------------------------#
# 计算测量误差
num_test_train = len(test_images)
        
layer_0 = conv_reshape(test_images, kernel_rows, kernel_cols)
layer_1 = conv_2d(layer_0, kernels, b_kernels)
layer_1 = layer_1.reshape(num_test_train,-1)
        
layer_2, pool_argmax = maxpool(layer_1, pool_rows, pool_cols, num_test_train, input_rows, input_cols, num_kernels)
layer_2 = layer_2.reshape(num_test_train, -1)
        
layer_3 = relu(np.dot(layer_2, w_2_3) + b_2_3)
layer_4 = softmax(np.dot(layer_3, w_3_4) + b_3_4)
        
test_loss, test_correct_cnt = 0, 0
for k in range(num_test_train):
    test_loss -= np.log(layer_4[k:k+1,np.argmax(test_labels[k:k+1])])
            
    test_correct_cnt += int(np.argmax(layer_4[k:k+1]) == np.argmax(test_labels[k:k+1]))
            
print("Test Loss: %0.3f Test Acc: %0.3f"% (test_loss/num_test_train, test_correct_cnt/num_test_train))